In [1]:
from simple_bank_korea.kb import get_transactions
import csv
import time
from ast import literal_eval
import paho.mqtt.client as mqtt
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [165]:
# example
transaction_list = get_transactions(
        bank_num='43140104100408',
        birthday='971028',
        password='1089',
        days=365                   #, default is 30
        # PHANTOM_PATH='/Users/beomi/bin/phantomjs', # Optional, default is 'phantomjs' only.
        # LOG_PATH='/Users/beomi/phantom.log' # Optional, default is os.path.devnull (no log
    )

::OS Detected - Windows::


# CSV 파일로 저장 및 불러오기

In [170]:
with open('bank_csv.csv','w',newline='',encoding = 'utf-8') as csvfile:
    writer = csv.DictWriter(csvfile,fieldnames = ['date','amount','balance','transaction_by'])
    writer.writeheader()
    for trs in transaction_list:
        writer.writerow(trs)

In [172]:
train = pd.read_csv("C:\\Users\\munan\\Desktop\\Fam_classification\\bank_csv.csv")

In [173]:
train

,date,amount,balance,transaction_by
0,2020-11-22 21:53:47,-1990,824,*10원캐쉬백주식회사 아성다
1,2020-11-22 21:27:44,1990,2814,KB체크할인
2,2020-11-21 13:34:20,-3700,824,현대할인마트
3,2020-11-21 02:15:48,-39800,4524,네이버페이
4,2020-11-21 02:15:24,40000,44324,강문영
...,...,...,...,...
95,2020-10-16 11:33:56,50000,70956,강문영
96,2020-10-15 18:42:32,-30500,20956,주꾸미
97,2020-10-15 18:42:27,50000,51456,이준수
98,2020-10-15 11:55:38,-4000,1456,쥬씨(죽전단국대점)


# 날짜 변수 달,월,날짜로 분리 및 추가

In [175]:
train['date'] = pd.to_datetime(train['date'])

In [176]:
train['date'].describe

<bound method NDFrame.describe of 0    2020-11-22 21:53:47
1    2020-11-22 21:27:44
2    2020-11-21 13:34:20
3    2020-11-21 02:15:48
4    2020-11-21 02:15:24
             ...        
95   2020-10-16 11:33:56
96   2020-10-15 18:42:32
97   2020-10-15 18:42:27
98   2020-10-15 11:55:38
99   2020-10-15 11:35:32
Name: date, Length: 100, dtype: datetime64[ns]>

In [177]:
train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month
train['day'] = train['date'].dt.day
train

,date,amount,balance,transaction_by,year,month,day
0,2020-11-22 21:53:47,-1990,824,*10원캐쉬백주식회사 아성다,2020,11,22
1,2020-11-22 21:27:44,1990,2814,KB체크할인,2020,11,22
2,2020-11-21 13:34:20,-3700,824,현대할인마트,2020,11,21
3,2020-11-21 02:15:48,-39800,4524,네이버페이,2020,11,21
4,2020-11-21 02:15:24,40000,44324,강문영,2020,11,21
...,...,...,...,...,...,...,...
95,2020-10-16 11:33:56,50000,70956,강문영,2020,10,16
96,2020-10-15 18:42:32,-30500,20956,주꾸미,2020,10,15
97,2020-10-15 18:42:27,50000,51456,이준수,2020,10,15
98,2020-10-15 11:55:38,-4000,1456,쥬씨(죽전단국대점),2020,10,15


# transaction_by 업종 별 분류 체계 필요 

In [178]:
train2 = train.copy()

In [219]:
## 일단 급해서 그냥 라벨러로 돌려버림
encoder = LabelEncoder()
train2['transaction_by'] = encoder.fit_transform(train2['transaction_by'])

In [180]:
train2

,date,amount,balance,transaction_by,year,month,day
0,2020-11-22 21:53:47,-1990,824,0,2020,11,22
1,2020-11-22 21:27:44,1990,2814,1,2020,11,22
2,2020-11-21 13:34:20,-3700,824,42,2020,11,21
3,2020-11-21 02:15:48,-39800,4524,5,2020,11,21
4,2020-11-21 02:15:24,40000,44324,2,2020,11,21
...,...,...,...,...,...,...,...
95,2020-10-16 11:33:56,50000,70956,2,2020,10,16
96,2020-10-15 18:42:32,-30500,20956,28,2020,10,15
97,2020-10-15 18:42:27,50000,51456,27,2020,10,15
98,2020-10-15 11:55:38,-4000,1456,29,2020,10,15


# 예측값이 balace 이기 때문에 balance 가 Y임

In [85]:
Y =train2['balance']

In [86]:
import os, json, glob, sys, numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, LSTM, Flatten, Dropout, Input, Conv1D, MaxPooling1D, Bidirectional, GlobalMaxPool1D
from keras.utils import np_utils
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras.backend.tensorflow_backend as K
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf

In [200]:
X = train2.drop('balance',1)
for i in range(len(X)):
    X['date'][i] = str(X['date'][i])

C:\Users\munan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\munan\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [201]:
a = X['date'].str.replace(pat=r'[^A-Za-z0-9]', repl=r' ', regex=True)

In [202]:
b = '-'.join(X['date'])

In [203]:
a[5]

'2020 11 19 18 17 11'

In [204]:
k=[]
for i in range(0,len(a)):
    k.append(float(a[i].replace(' ','')[4:-2])/1000)

In [205]:
X['date'] = k
X

,date,amount,transaction_by,year,month,day
0,11222.153,-1990,*10원캐쉬백주식회사 아성다,2020,11,22
1,11222.127,1990,KB체크할인,2020,11,22
2,11211.334,-3700,현대할인마트,2020,11,21
3,11210.215,-39800,네이버페이,2020,11,21
4,11210.215,40000,강문영,2020,11,21
...,...,...,...,...,...,...
115,10161.133,50000,강문영,2020,10,16
116,10151.842,-30500,주꾸미,2020,10,15
117,10151.842,50000,이준수,2020,10,15
118,10151.155,-4000,쥬씨(죽전단국대점),2020,10,15


In [93]:
X= np.array(X)

In [71]:
X.shape

(100, 6)

In [72]:
y_train.shape

(80, 1)

In [142]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

# LSTM 모델

In [131]:
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential

In [242]:
model = Sequential()

model.add(Dense(units=1024, input_dim=6, activation='relu'))
model.add(Dense(units=765, activation='relu'))
model.add(Dense(units=612, activation='relu'))
model.add(Dense(units=512, activation='relu'))
model.add(Dense(units=400, activation='relu'))
model.add(Dense(units=356, activation='relu'))
model.add(Dense(units=300, activation='relu'))
model.add(Dense(units=256, activation='relu'))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=1))

# 모델 학습과정 설정 
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [143]:
# 학습 조기종료 
from keras.callbacks import EarlyStopping
es = EarlyStopping(patience=10)

# 모델 트레이닝 
hist = model.fit(X_train, y_train, epochs=1000, batch_size=16, callbacks=[es])

Epoch 1/1000
80/80 [==============================] - 0s 237us/step - loss: 3155464435.2000 - mae: 27758.5938
Epoch 2/1000
80/80 [==============================] - 0s 199us/step - loss: 6165342848.0000 - mae: 36487.5703
Epoch 3/1000
80/80 [==============================] - 0s 199us/step - loss: 4685583475.2000 - mae: 34594.4297
Epoch 4/1000
80/80 [==============================] - 0s 224us/step - loss: 2359806566.4000 - mae: 33502.2383
Epoch 5/1000
80/80 [==============================] - 0s 237us/step - loss: 4041606502.4000 - mae: 37282.9453
Epoch 6/1000
80/80 [==============================] - 0s 249us/step - loss: 3162109440.0000 - mae: 39080.7266
Epoch 7/1000
80/80 [==============================] - 0s 237us/step - loss: 2541035379.2000 - mae: 34223.6016
Epoch 8/1000
80/80 [==============================] - 0s 237us/step - loss: 2031521408.0000 - mae: 32587.2539
Epoch 9/1000
80/80 [==============================] - 0s 212us/step - loss: 1598874931.2000 - mae: 30161.6562
Epoch 10/1

C:\Users\munan\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,mae
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Epoch 12/1000
80/80 [==============================] - 0s 262us/step - loss: 1709283596.8000 - mae: 28522.5625
Epoch 13/1000
80/80 [==============================] - 0s 224us/step - loss: 1305630284.8000 - mae: 26388.2852
Epoch 14/1000
80/80 [==============================] - 0s 224us/step - loss: 5816275916.8000 - mae: 33125.2188
Epoch 15/1000
80/80 [==============================] - 0s 237us/step - loss: 4156697318.4000 - mae: 33060.7773
Epoch 16/1000
80/80 [==============================] - 0s 224us/step - loss: 4638392729.6000 - mae: 33910.9023
Epoch 17/1000
80/80 [==============================] - 0s 237us/step - loss: 3232719257.6000 - mae: 30698.2031
Epoch 18/1000
80/80 [==============================] - 0s 237us/step - loss: 1140153574.4000 - mae: 25253.0898
Epoch 19/1000
80/80 [==============================] - 0s 212us/step - loss: 1861931417.6000 - mae: 27632.9316
Epoch 20/1000
80/80 [==============================] - 0s 212us/step - loss: 1300518528.0000 - mae: 26235.3320
E

80/80 [==============================] - 0s 212us/step - loss: 585994553.6000 - mae: 17785.3008
Epoch 160/1000
80/80 [==============================] - 0s 212us/step - loss: 829111513.6000 - mae: 20329.8047
Epoch 161/1000
80/80 [==============================] - 0s 212us/step - loss: 681957568.0000 - mae: 18874.3262
Epoch 162/1000
80/80 [==============================] - 0s 199us/step - loss: 753374560.0000 - mae: 19502.8047
Epoch 163/1000
80/80 [==============================] - 0s 212us/step - loss: 703375072.0000 - mae: 19391.6680
Epoch 164/1000
80/80 [==============================] - 0s 199us/step - loss: 827596947.2000 - mae: 19516.2500
Epoch 165/1000
80/80 [==============================] - 0s 187us/step - loss: 789230912.0000 - mae: 19788.7305
Epoch 166/1000
80/80 [==============================] - 0s 212us/step - loss: 927478310.4000 - mae: 20545.6367
Epoch 167/1000
80/80 [==============================] - 0s 199us/step - loss: 982824422.4000 - mae: 21682.4023
Epoch 168/1000
8

80/80 [==============================] - 0s 199us/step - loss: 989143507.2000 - mae: 20555.8262
Epoch 306/1000
80/80 [==============================] - 0s 212us/step - loss: 1024891296.0000 - mae: 20591.7773
Epoch 307/1000
80/80 [==============================] - 0s 237us/step - loss: 991868832.0000 - mae: 20368.4629
Epoch 308/1000
80/80 [==============================] - 0s 212us/step - loss: 1008399660.8000 - mae: 21640.0059
Epoch 309/1000
80/80 [==============================] - 0s 212us/step - loss: 984635961.6000 - mae: 19886.0410
Epoch 310/1000
80/80 [==============================] - 0s 212us/step - loss: 896379808.0000 - mae: 18842.0449
Epoch 311/1000
80/80 [==============================] - 0s 187us/step - loss: 894464569.6000 - mae: 19300.2070
Epoch 312/1000
80/80 [==============================] - 0s 199us/step - loss: 871636499.2000 - mae: 18622.7070
Epoch 313/1000
80/80 [==============================] - 0s 212us/step - loss: 878485651.2000 - mae: 18993.6289
Epoch 314/1000

80/80 [==============================] - 0s 212us/step - loss: 822553814.4000 - mae: 18043.9180
Epoch 452/1000
80/80 [==============================] - 0s 224us/step - loss: 979498912.0000 - mae: 20891.0566
Epoch 453/1000
80/80 [==============================] - 0s 249us/step - loss: 873045612.8000 - mae: 19645.8496
Epoch 454/1000
80/80 [==============================] - 0s 224us/step - loss: 838441196.8000 - mae: 18792.0781
Epoch 455/1000
80/80 [==============================] - 0s 212us/step - loss: 929574662.4000 - mae: 18992.1895
Epoch 456/1000
80/80 [==============================] - 0s 212us/step - loss: 889856448.0000 - mae: 19466.0371
Epoch 457/1000
80/80 [==============================] - 0s 224us/step - loss: 870100281.6000 - mae: 18738.5820
Epoch 458/1000
80/80 [==============================] - 0s 212us/step - loss: 1018165145.6000 - mae: 20380.8984
Epoch 459/1000
80/80 [==============================] - 0s 224us/step - loss: 1028374028.8000 - mae: 20853.8398
Epoch 460/1000

80/80 [==============================] - 0s 199us/step - loss: 942112038.4000 - mae: 18946.8105
Epoch 598/1000
80/80 [==============================] - 0s 199us/step - loss: 1245958316.8000 - mae: 22240.4727
Epoch 599/1000
80/80 [==============================] - 0s 212us/step - loss: 1830192576.0000 - mae: 28866.5195
Epoch 600/1000
80/80 [==============================] - 0s 224us/step - loss: 1696164800.0000 - mae: 28515.4375
Epoch 601/1000
80/80 [==============================] - 0s 237us/step - loss: 1603708595.2000 - mae: 29625.6133
Epoch 602/1000
80/80 [==============================] - 0s 212us/step - loss: 1542761369.6000 - mae: 28918.0410
Epoch 603/1000
80/80 [==============================] - 0s 224us/step - loss: 1520880691.2000 - mae: 27126.5723
Epoch 604/1000
80/80 [==============================] - 0s 212us/step - loss: 1389683840.0000 - mae: 25479.1504
Epoch 605/1000
80/80 [==============================] - 0s 212us/step - loss: 1355950336.0000 - mae: 25307.0020
Epoch 60

80/80 [==============================] - 0s 212us/step - loss: 1799419968.0000 - mae: 26158.3691
Epoch 744/1000
80/80 [==============================] - 0s 212us/step - loss: 1464795244.8000 - mae: 25087.9102
Epoch 745/1000
80/80 [==============================] - 0s 199us/step - loss: 1362681177.6000 - mae: 23741.9492
Epoch 746/1000
80/80 [==============================] - 0s 187us/step - loss: 1286154163.2000 - mae: 24104.8750
Epoch 747/1000
80/80 [==============================] - 0s 199us/step - loss: 1281930713.6000 - mae: 22790.5781
Epoch 748/1000
80/80 [==============================] - 0s 187us/step - loss: 1226526835.2000 - mae: 21182.2383
Epoch 749/1000
80/80 [==============================] - 0s 199us/step - loss: 1112157715.2000 - mae: 21576.7461
Epoch 750/1000
80/80 [==============================] - 0s 212us/step - loss: 1085615782.4000 - mae: 21395.5508
Epoch 751/1000
80/80 [==============================] - 0s 199us/step - loss: 1222844320.0000 - mae: 22263.6191
Epoch 7

80/80 [==============================] - 0s 199us/step - loss: 1007734761.6000 - mae: 21040.8418
Epoch 890/1000
80/80 [==============================] - 0s 249us/step - loss: 1161256396.8000 - mae: 21673.8730
Epoch 891/1000
80/80 [==============================] - 0s 237us/step - loss: 930402726.4000 - mae: 18902.2988
Epoch 892/1000
80/80 [==============================] - 0s 224us/step - loss: 965762176.0000 - mae: 20442.2910
Epoch 893/1000
80/80 [==============================] - 0s 212us/step - loss: 989671715.2000 - mae: 19419.6641
Epoch 894/1000
80/80 [==============================] - 0s 224us/step - loss: 1137525286.4000 - mae: 21381.8887
Epoch 895/1000
80/80 [==============================] - 0s 212us/step - loss: 953708787.2000 - mae: 19999.1562
Epoch 896/1000
80/80 [==============================] - 0s 199us/step - loss: 1141817651.2000 - mae: 19140.0586
Epoch 897/1000
80/80 [==============================] - 0s 212us/step - loss: 1159310054.4000 - mae: 20879.6211
Epoch 898/1

In [144]:
res = model.evaluate(X_test, y_test, batch_size=32)
print("loss",res[0],"mae",res[1])

20/20 [==============================] - 0s 100us/step
loss 3382695168.0 mae 46744.7890625


In [145]:
y_pred=model.predict(X_test)

In [146]:
y_pred = list(map(int,y_pred))

In [147]:
y_pred

[55971,
 66644,
 110984,
 37917,
 157752,
 56678,
 56288,
 221782,
 56719,
 106249,
 95048,
 41873,
 76027,
 128065,
 137665,
 17676,
 34085,
 68506,
 89492,
 103543]

In [148]:
y_pred = pd.Series(y_pred)

# 이대로 쓸 경우 잔고 예측(소비 데이터 하위 20% copy및 학습 사용)

In [211]:
train2 = train[-20:]

In [212]:
train2 = pd.concat([train,train2],ignore_index =True)
train2

,date,amount,balance,transaction_by,year,month,day
0,2020-11-22 21:53:47,-1990,824,*10원캐쉬백주식회사 아성다,2020,11,22
1,2020-11-22 21:27:44,1990,2814,KB체크할인,2020,11,22
2,2020-11-21 13:34:20,-3700,824,현대할인마트,2020,11,21
3,2020-11-21 02:15:48,-39800,4524,네이버페이,2020,11,21
4,2020-11-21 02:15:24,40000,44324,강문영,2020,11,21
...,...,...,...,...,...,...,...
115,2020-10-16 11:33:56,50000,70956,강문영,2020,10,16
116,2020-10-15 18:42:32,-30500,20956,주꾸미,2020,10,15
117,2020-10-15 18:42:27,50000,51456,이준수,2020,10,15
118,2020-10-15 11:55:38,-4000,1456,쥬씨(죽전단국대점),2020,10,15


In [213]:
Y=train2['balance']
X = train2.drop('balance',1)

In [214]:
X = train2.drop('balance',1)
for i in range(len(X)):
    X['date'][i] = str(X['date'][i])

C:\Users\munan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\munan\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [215]:

k=[]
for i in range(0,len(a)):
    k.append(float(a[i].replace(' ','')[4:-2])/1000)
    
X['date'] = k
X

,date,amount,transaction_by,year,month,day
0,11222.153,-1990,*10원캐쉬백주식회사 아성다,2020,11,22
1,11222.127,1990,KB체크할인,2020,11,22
2,11211.334,-3700,현대할인마트,2020,11,21
3,11210.215,-39800,네이버페이,2020,11,21
4,11210.215,40000,강문영,2020,11,21
...,...,...,...,...,...,...
115,10161.133,50000,강문영,2020,10,16
116,10151.842,-30500,주꾸미,2020,10,15
117,10151.842,50000,이준수,2020,10,15
118,10151.155,-4000,쥬씨(죽전단국대점),2020,10,15


In [220]:
## 일단 급해서 그냥 라벨러로 돌려버림
encoder = LabelEncoder()
X['transaction_by'] = encoder.fit_transform(X['transaction_by'])

In [221]:
x_train = X[:-20]
x_test = X[-20:]
y_train = Y[:-20]
y_test = Y[-20:]

In [222]:
# 학습 조기종료 
from keras.callbacks import EarlyStopping
es = EarlyStopping(patience=10)

# 모델 트레이닝 
hist = model.fit(x_train, y_train, epochs=1000, batch_size=16, callbacks=[es])

Epoch 1/1000
100/100 [==============================] - 0s 309us/step - loss: 1759673344.0000 - mae: 28608.4180
Epoch 2/1000
100/100 [==============================] - 0s 259us/step - loss: 1882504980.4800 - mae: 27187.8730
Epoch 3/1000
100/100 [==============================] - 0s 259us/step - loss: 1986771187.2000 - mae: 28695.4844
Epoch 4/1000
100/100 [==============================] - 0s 249us/step - loss: 2290165580.8000 - mae: 33416.7344
Epoch 5/1000
100/100 [==============================] - 0s 269us/step - loss: 1917124367.3600 - mae: 30494.3027
Epoch 6/1000
100/100 [==============================] - 0s 259us/step - loss: 1775059732.4800 - mae: 30049.7246
Epoch 7/1000
100/100 [==============================] - 0s 280us/step - loss: 2120651330.5600 - mae: 30506.3203
Epoch 8/1000
100/100 [==============================] - 0s 229us/step - loss: 1809034608.6400 - mae: 29483.0820


C:\Users\munan\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,mae
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Epoch 9/1000
100/100 [==============================] - 0s 239us/step - loss: 1571019671.0400 - mae: 27234.1777
Epoch 10/1000
100/100 [==============================] - 0s 289us/step - loss: 1495428852.4800 - mae: 26273.1367
Epoch 11/1000
100/100 [==============================] - 0s 229us/step - loss: 1701396413.4400 - mae: 26114.7344
Epoch 12/1000
100/100 [==============================] - 0s 269us/step - loss: 1257505448.9600 - mae: 24276.8926
Epoch 13/1000
100/100 [==============================] - 0s 259us/step - loss: 1405732111.3600 - mae: 25364.3770
Epoch 14/1000
100/100 [==============================] - 0s 239us/step - loss: 2047963065.6000 - mae: 25603.8242
Epoch 15/1000
100/100 [==============================] - 0s 229us/step - loss: 1474721927.6800 - mae: 25436.6152
Epoch 16/1000
100/100 [==============================] - 0s 229us/step - loss: 1429043657.6000 - mae: 26096.8984
Epoch 17/1000
100/100 [==============================] - 0s 259us/step - loss: 1227564037.1200 - 

100/100 [==============================] - 0s 259us/step - loss: 3268916897.2800 - mae: 35142.2031
Epoch 82/1000
100/100 [==============================] - 0s 219us/step - loss: 3771080153.6000 - mae: 38566.6172
Epoch 83/1000
100/100 [==============================] - 0s 249us/step - loss: 2540058408.9600 - mae: 35044.0781
Epoch 84/1000
100/100 [==============================] - 0s 249us/step - loss: 4486829568.0000 - mae: 39648.3164
Epoch 85/1000
100/100 [==============================] - 0s 259us/step - loss: 3451723845.1200 - mae: 38328.1367
Epoch 86/1000
100/100 [==============================] - 0s 240us/step - loss: 2826930114.5600 - mae: 35936.4883
Epoch 87/1000
100/100 [==============================] - 0s 245us/step - loss: 2360153953.2800 - mae: 32208.1094
Epoch 88/1000
100/100 [==============================] - 0s 239us/step - loss: 2365687521.2800 - mae: 32290.9902
Epoch 89/1000
100/100 [==============================] - 0s 239us/step - loss: 2330896163.8400 - mae: 32649.64

Epoch 225/1000
100/100 [==============================] - 0s 239us/step - loss: 2038262050.5600 - mae: 28452.0859
Epoch 226/1000
100/100 [==============================] - 0s 249us/step - loss: 2084161341.4400 - mae: 29745.1816
Epoch 227/1000
100/100 [==============================] - 0s 255us/step - loss: 2116810506.2400 - mae: 29634.4707
Epoch 228/1000
100/100 [==============================] - 0s 259us/step - loss: 2045544232.9600 - mae: 27970.3926
Epoch 229/1000
100/100 [==============================] - 0s 237us/step - loss: 2206909032.9600 - mae: 28940.6504
Epoch 230/1000
100/100 [==============================] - 0s 259us/step - loss: 2254909016.3200 - mae: 29481.1582
Epoch 231/1000
100/100 [==============================] - 0s 237us/step - loss: 2026935685.1200 - mae: 28179.6992
Epoch 232/1000
100/100 [==============================] - 0s 269us/step - loss: 2287782364.1600 - mae: 32246.7383
Epoch 233/1000
100/100 [==============================] - 0s 249us/step - loss: 26882904

Epoch 368/1000
100/100 [==============================] - 0s 229us/step - loss: 1998802534.4000 - mae: 30961.3301
Epoch 369/1000
100/100 [==============================] - 0s 249us/step - loss: 1685090092.8000 - mae: 26994.3418
Epoch 370/1000
100/100 [==============================] - 0s 249us/step - loss: 1706230446.0800 - mae: 27693.5547
Epoch 371/1000
100/100 [==============================] - 0s 279us/step - loss: 1926520316.1600 - mae: 30084.6133
Epoch 372/1000
100/100 [==============================] - 0s 229us/step - loss: 1666399388.1600 - mae: 28979.9043
Epoch 373/1000
100/100 [==============================] - 0s 249us/step - loss: 2122363392.0000 - mae: 27163.5645
Epoch 374/1000
100/100 [==============================] - 0s 239us/step - loss: 1849814586.8800 - mae: 27188.4199
Epoch 375/1000
100/100 [==============================] - 0s 247us/step - loss: 1684828211.2000 - mae: 28480.7441
Epoch 376/1000
100/100 [==============================] - 0s 229us/step - loss: 16258264

Epoch 511/1000
100/100 [==============================] - 0s 279us/step - loss: 1699634749.4400 - mae: 28959.7734
Epoch 512/1000
100/100 [==============================] - 0s 289us/step - loss: 1672299432.9600 - mae: 27013.2207
Epoch 513/1000
100/100 [==============================] - 0s 229us/step - loss: 1794628149.7600 - mae: 28530.4844
Epoch 514/1000
100/100 [==============================] - 0s 259us/step - loss: 1632301061.1200 - mae: 28012.1152
Epoch 515/1000
100/100 [==============================] - 0s 219us/step - loss: 1618711183.3600 - mae: 28020.0566
Epoch 516/1000
100/100 [==============================] - 0s 269us/step - loss: 1874719467.5200 - mae: 27601.9219
Epoch 517/1000
100/100 [==============================] - 0s 229us/step - loss: 2452688591.3600 - mae: 30134.6680
Epoch 518/1000
100/100 [==============================] - 0s 269us/step - loss: 2379014973.4400 - mae: 30163.5879
Epoch 519/1000
100/100 [==============================] - 0s 269us/step - loss: 23288921

Epoch 654/1000
100/100 [==============================] - 0s 239us/step - loss: 1352087616.0000 - mae: 25762.8691
Epoch 655/1000
100/100 [==============================] - 0s 259us/step - loss: 2434241955.8400 - mae: 30101.5781
Epoch 656/1000
100/100 [==============================] - 0s 239us/step - loss: 2343960161.2800 - mae: 28202.3945
Epoch 657/1000
100/100 [==============================] - 0s 219us/step - loss: 2458915929.6000 - mae: 29980.5742
Epoch 658/1000
100/100 [==============================] - 0s 249us/step - loss: 2418314938.2400 - mae: 28839.2266
Epoch 659/1000
100/100 [==============================] - 0s 229us/step - loss: 2333580006.4000 - mae: 27741.8320
Epoch 660/1000
100/100 [==============================] - 0s 259us/step - loss: 2305794181.1200 - mae: 27737.7695
Epoch 661/1000
100/100 [==============================] - 0s 259us/step - loss: 2317573071.3600 - mae: 27654.6680
Epoch 662/1000
100/100 [==============================] - 0s 229us/step - loss: 23158362

Epoch 797/1000
100/100 [==============================] - 0s 259us/step - loss: 1359300720.6400 - mae: 23602.8105
Epoch 798/1000
100/100 [==============================] - 0s 239us/step - loss: 1121136798.7200 - mae: 23092.8809
Epoch 799/1000
100/100 [==============================] - 0s 259us/step - loss: 1086486844.1600 - mae: 23315.1582
Epoch 800/1000
100/100 [==============================] - 0s 305us/step - loss: 1245542336.6400 - mae: 24133.1973
Epoch 801/1000
100/100 [==============================] - 0s 240us/step - loss: 1029081149.4400 - mae: 22735.3203
Epoch 802/1000
100/100 [==============================] - 0s 289us/step - loss: 1328029532.1600 - mae: 25061.7148
Epoch 803/1000
100/100 [==============================] - 0s 259us/step - loss: 1364941973.7600 - mae: 23863.3594
Epoch 804/1000
100/100 [==============================] - 0s 265us/step - loss: 1278262732.8000 - mae: 24529.0703
Epoch 805/1000
100/100 [==============================] - 0s 229us/step - loss: 14035965

Epoch 940/1000
100/100 [==============================] - 0s 219us/step - loss: 2072656885.7600 - mae: 29307.9199
Epoch 941/1000
100/100 [==============================] - 0s 259us/step - loss: 2131664936.9600 - mae: 30663.3945
Epoch 942/1000
100/100 [==============================] - 0s 229us/step - loss: 3030582538.2400 - mae: 35284.5352
Epoch 943/1000
100/100 [==============================] - 0s 249us/step - loss: 2293348543.3600 - mae: 32237.8516
Epoch 944/1000
100/100 [==============================] - 0s 249us/step - loss: 2170855518.7200 - mae: 31419.3008
Epoch 945/1000
100/100 [==============================] - 0s 279us/step - loss: 1973162229.7600 - mae: 29577.2773
Epoch 946/1000
100/100 [==============================] - 0s 229us/step - loss: 1814604830.7200 - mae: 28396.2246
Epoch 947/1000
100/100 [==============================] - ETA: 0s - loss: 2446558464.0000 - mae: 31094.81 - 0s 279us/step - loss: 1650637025.2800 - mae: 26511.6797
Epoch 948/1000
100/100 [==============

In [223]:
res = model.evaluate(x_test, y_test, batch_size=32)
print("loss",res[0],"mae",res[1])

20/20 [==============================] - 0s 100us/step
loss 680952640.0 mae 21353.677734375


In [226]:
y_pred=model.predict(x_test)
y_pred = list(map(int,y_pred))
y_pred[-7:]

[39322, 53712, 75785, 71197, 73633, 45946, 40332]

# 아껴서 쓰면 다음주 잔고 예측

In [234]:
a = train2['amount']
k=[]
for i in range(0,len(a)):
    k.append(a[i]//5) ## 5% 아낀다고 가정
train2['amount'] =k

[-398,
 398,
 -740,
 -7960,
 8000,
 -1480,
 -2280,
 -1400,
 -720,
 -2234,
 -2100,
 -2060,
 -800,
 -2660,
 -900,
 -1520,
 -2400,
 -1394,
 -720,
 10000,
 10000,
 -1640,
 -1420,
 -860,
 -3280,
 -760,
 -960,
 -1600,
 -760,
 -400,
 -2800,
 -860,
 -3500,
 10000,
 10000,
 -1376,
 -1640,
 -960,
 -400,
 -3600,
 -1419,
 -1960,
 -3500,
 -2160,
 -796,
 10000,
 10000,
 -760,
 -1800,
 -1500,
 -300,
 -900,
 -1440,
 -1500,
 -1000,
 -4400,
 -2200,
 -2060,
 -53000,
 -6500,
 -60000,
 113120,
 -3600,
 -2200,
 -2800,
 -900,
 1200,
 -1200,
 -760,
 6000,
 -400,
 4000,
 -960,
 20000,
 -500,
 -1040,
 -2800,
 -1960,
 10000,
 -400,
 -560,
 -2400,
 -1000,
 -760,
 -640,
 -12000,
 10000,
 -620,
 -2228,
 -1360,
 10000,
 -640,
 -6000,
 -1080,
 -2160,
 10000,
 -6100,
 10000,
 -800,
 -4140,
 -560,
 -2400,
 -1000,
 -760,
 -640,
 -12000,
 10000,
 -620,
 -2228,
 -1360,
 10000,
 -640,
 -6000,
 -1080,
 -2160,
 10000,
 -6100,
 10000,
 -800,
 -4140]

In [235]:
train2['amount'] =k

In [236]:
train2

,date,amount,balance,transaction_by,year,month,day
0,2020-11-22 21:53:47,-398,824,0,2020,11,22
1,2020-11-22 21:27:44,398,2814,1,2020,11,22
2,2020-11-21 13:34:20,-740,824,42,2020,11,21
3,2020-11-21 02:15:48,-7960,4524,5,2020,11,21
4,2020-11-21 02:15:24,8000,44324,2,2020,11,21
...,...,...,...,...,...,...,...
115,2020-10-16 11:33:56,10000,70956,2,2020,10,16
116,2020-10-15 18:42:32,-6100,20956,28,2020,10,15
117,2020-10-15 18:42:27,10000,51456,27,2020,10,15
118,2020-10-15 11:55:38,-800,1456,29,2020,10,15


In [237]:
x_train = X[:-20]
x_test = X[-20:]
y_train = Y[:-20]
y_test = Y[-20:]

In [238]:
# 학습 조기종료 
from keras.callbacks import EarlyStopping
es = EarlyStopping(patience=10)

# 모델 트레이닝 
hist = model.fit(x_train, y_train, epochs=1000, batch_size=16, callbacks=[es])

Epoch 1/1000
100/100 [==============================] - 0s 269us/step - loss: 2286846996.4800 - mae: 29822.0566
Epoch 2/1000
100/100 [==============================] - 0s 250us/step - loss: 2318466990.0800 - mae: 32026.6191
Epoch 3/1000
100/100 [==============================] - 0s 230us/step - loss: 2418889277.4400 - mae: 32050.8828
Epoch 4/1000
100/100 [==============================] - 0s 319us/step - loss: 2437094082.2400 - mae: 31173.6758
Epoch 5/1000
100/100 [==============================] - 0s 299us/step - loss: 2301107983.3600 - mae: 31777.8027
Epoch 6/1000
100/100 [==============================] - 0s 279us/step - loss: 2277910712.3200 - mae: 31840.2891
Epoch 7/1000
100/100 [==============================] - 0s 270us/step - loss: 2177886151.6800 - mae: 30552.2246
Epoch 8/1000
100/100 [==============================] - 0s 239us/step - loss: 2212004359.6800 - mae: 30280.8066


C:\Users\munan\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,mae
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Epoch 9/1000
100/100 [==============================] - 0s 269us/step - loss: 2179479883.5200 - mae: 29570.4043
Epoch 10/1000
100/100 [==============================] - 0s 261us/step - loss: 2132851118.0800 - mae: 29710.2793
Epoch 11/1000
100/100 [==============================] - 0s 259us/step - loss: 2102117867.5200 - mae: 29208.3945
Epoch 12/1000
100/100 [==============================] - 0s 269us/step - loss: 2001596226.5600 - mae: 28222.5176
Epoch 13/1000
100/100 [==============================] - 0s 269us/step - loss: 2225944042.8800 - mae: 28961.6934
Epoch 14/1000
100/100 [==============================] - 0s 240us/step - loss: 2124535326.7200 - mae: 28204.4219
Epoch 15/1000
100/100 [==============================] - 0s 280us/step - loss: 2066856588.8000 - mae: 28909.0176
Epoch 16/1000
100/100 [==============================] - 0s 259us/step - loss: 2076340602.5600 - mae: 28773.0547
Epoch 17/1000
100/100 [==============================] - 0s 230us/step - loss: 1943316602.8800 - 

100/100 [==============================] - 0s 236us/step - loss: 1899534776.3200 - mae: 27300.3184
Epoch 154/1000
100/100 [==============================] - 0s 250us/step - loss: 1862099712.0000 - mae: 26639.4395
Epoch 155/1000
100/100 [==============================] - 0s 219us/step - loss: 1842993935.3600 - mae: 26578.9102
Epoch 156/1000
100/100 [==============================] - 0s 240us/step - loss: 1841644966.4000 - mae: 26777.9375
Epoch 157/1000
100/100 [==============================] - 0s 259us/step - loss: 1769244784.6400 - mae: 27328.0391
Epoch 158/1000
100/100 [==============================] - 0s 239us/step - loss: 2251237488.6400 - mae: 30793.4023
Epoch 159/1000
100/100 [==============================] - 0s 239us/step - loss: 2324814860.8000 - mae: 31428.6250
Epoch 160/1000
100/100 [==============================] - 0s 279us/step - loss: 2138756044.8000 - mae: 31075.1934
Epoch 161/1000
100/100 [==============================] - 0s 230us/step - loss: 2030187130.8800 - mae: 

Epoch 225/1000
100/100 [==============================] - 0s 279us/step - loss: 1831987906.5600 - mae: 28737.2500
Epoch 226/1000
100/100 [==============================] - 0s 309us/step - loss: 1918107612.1600 - mae: 31223.5098
Epoch 227/1000
100/100 [==============================] - 0s 259us/step - loss: 1479211601.9200 - mae: 25730.8828
Epoch 228/1000
100/100 [==============================] - 0s 259us/step - loss: 1654063868.1600 - mae: 27344.6934
Epoch 229/1000
100/100 [==============================] - 0s 239us/step - loss: 1616705561.6000 - mae: 27097.9375
Epoch 230/1000
100/100 [==============================] - 0s 229us/step - loss: 1517811051.5200 - mae: 26183.5391
Epoch 231/1000
100/100 [==============================] - 0s 279us/step - loss: 1450690201.6000 - mae: 25452.8730
Epoch 232/1000
100/100 [==============================] - 0s 259us/step - loss: 1632442900.4800 - mae: 25521.0547
Epoch 233/1000
100/100 [==============================] - 0s 239us/step - loss: 17235931

Epoch 368/1000
100/100 [==============================] - 0s 279us/step - loss: 1533701483.5200 - mae: 26603.4805
Epoch 369/1000
100/100 [==============================] - 0s 229us/step - loss: 1542521338.8800 - mae: 25835.2598
Epoch 370/1000
100/100 [==============================] - 0s 259us/step - loss: 1609296829.4400 - mae: 27598.6543
Epoch 371/1000
100/100 [==============================] - 0s 269us/step - loss: 1536290099.2000 - mae: 26040.1699
Epoch 372/1000
100/100 [==============================] - 0s 220us/step - loss: 1527252408.3200 - mae: 26457.2695
Epoch 373/1000
100/100 [==============================] - 0s 290us/step - loss: 1570564407.0400 - mae: 26345.9355
Epoch 374/1000
100/100 [==============================] - 0s 259us/step - loss: 1516334274.5600 - mae: 26125.9668
Epoch 375/1000
100/100 [==============================] - 0s 229us/step - loss: 1599474557.4400 - mae: 27414.8027
Epoch 376/1000
100/100 [==============================] - 0s 261us/step - loss: 15033380

100/100 [==============================] - 0s 249us/step - loss: 1610198952.9600 - mae: 27972.8926
Epoch 511/1000
100/100 [==============================] - 0s 269us/step - loss: 1423597148.1600 - mae: 26060.4375
Epoch 512/1000
100/100 [==============================] - 0s 269us/step - loss: 1241455185.9200 - mae: 24705.0332
Epoch 513/1000
100/100 [==============================] - 0s 230us/step - loss: 1254987471.3600 - mae: 23511.4941
Epoch 514/1000
100/100 [==============================] - 0s 220us/step - loss: 1415223307.5200 - mae: 25618.5234
Epoch 515/1000
100/100 [==============================] - 0s 229us/step - loss: 1258415687.6800 - mae: 24940.5547
Epoch 516/1000
100/100 [==============================] - 0s 259us/step - loss: 1125275169.2800 - mae: 24252.3105
Epoch 517/1000
100/100 [==============================] - 0s 219us/step - loss: 1269194570.2400 - mae: 24067.2207
Epoch 518/1000
100/100 [==============================] - 0s 230us/step - loss: 1316125358.0800 - mae: 

100/100 [==============================] - 0s 289us/step - loss: 1073078010.8800 - mae: 22151.8652
Epoch 653/1000
100/100 [==============================] - 0s 269us/step - loss: 901611585.2800 - mae: 20753.0332
Epoch 654/1000
100/100 [==============================] - 0s 289us/step - loss: 894162558.7200 - mae: 20988.2383
Epoch 655/1000
100/100 [==============================] - 0s 239us/step - loss: 850997862.4000 - mae: 20366.7910
Epoch 656/1000
100/100 [==============================] - 0s 279us/step - loss: 828044672.0000 - mae: 20232.7578
Epoch 657/1000
100/100 [==============================] - 0s 249us/step - loss: 816754408.9600 - mae: 19818.9395
Epoch 658/1000
100/100 [==============================] - 0s 240us/step - loss: 1033259223.0400 - mae: 21916.8496
Epoch 659/1000
100/100 [==============================] - 0s 249us/step - loss: 1172998512.6400 - mae: 24666.2598
Epoch 660/1000
100/100 [==============================] - 0s 260us/step - loss: 1317142104.3200 - mae: 24268

Epoch 796/1000
100/100 [==============================] - 0s 239us/step - loss: 1510068439.0400 - mae: 26701.8496
Epoch 797/1000
100/100 [==============================] - 0s 259us/step - loss: 1513504890.8800 - mae: 25889.8008
Epoch 798/1000
100/100 [==============================] - 0s 269us/step - loss: 1407031623.6800 - mae: 24934.2305
Epoch 799/1000
100/100 [==============================] - 0s 230us/step - loss: 1489540270.0800 - mae: 26480.4043
Epoch 800/1000
100/100 [==============================] - 0s 230us/step - loss: 1932021427.2000 - mae: 28682.0469
Epoch 801/1000
100/100 [==============================] - 0s 270us/step - loss: 1542351040.0000 - mae: 26840.7070
Epoch 802/1000
100/100 [==============================] - 0s 219us/step - loss: 1558796149.7600 - mae: 26017.7344
Epoch 803/1000
100/100 [==============================] - 0s 240us/step - loss: 1525992600.9600 - mae: 25422.1133
Epoch 804/1000
100/100 [==============================] - 0s 270us/step - loss: 12996663

Epoch 939/1000
100/100 [==============================] - 0s 239us/step - loss: 1933383895.0400 - mae: 25720.0820
Epoch 940/1000
100/100 [==============================] - 0s 229us/step - loss: 1974738647.0400 - mae: 26641.6133
Epoch 941/1000
100/100 [==============================] - 0s 279us/step - loss: 1915166336.0000 - mae: 25833.7148
Epoch 942/1000
100/100 [==============================] - 0s 259us/step - loss: 2020283975.6800 - mae: 26350.5781
Epoch 943/1000
100/100 [==============================] - 0s 249us/step - loss: 1901191613.4400 - mae: 26329.0859
Epoch 944/1000
100/100 [==============================] - 0s 229us/step - loss: 1691760156.1600 - mae: 25610.1016
Epoch 945/1000
100/100 [==============================] - 0s 259us/step - loss: 1681887150.0800 - mae: 26716.6250
Epoch 946/1000
100/100 [==============================] - 0s 229us/step - loss: 1415987496.9600 - mae: 25256.6953
Epoch 947/1000
100/100 [==============================] - 0s 239us/step - loss: 14598821

In [239]:
res = model.evaluate(x_test, y_test, batch_size=32)
print("loss",res[0],"mae",res[1])

20/20 [==============================] - 0s 100us/step
loss 1380438016.0 mae 29791.03515625


In [241]:
y_pred=model.predict(x_test)
y_pred = list(map(int,y_pred))
y_pred[-20:]


[6144,
 61993,
 66132,
 84898,
 34651,
 15271,
 15515,
 33285,
 50435,
 68899,
 13724,
 33768,
 48818,
 31994,
 55350,
 15492,
 64289,
 13702,
 47146,
 17849]